In [53]:
# Клонируем репозиторий и переходим в папку проекта
!git clone https://github.com/dejar212/Diffraction.git
%cd Diffraction


Cloning into 'Diffraction'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 95 (delta 2), reused 7 (delta 2), pack-reused 79 (from 1)
Receiving objects: 100% (95/95), 511.01 KiB | 14.60 MiB/s, done.
Resolving deltas: 100% (22/22), done.
/content/Diffraction/Diffraction


In [54]:
%cd /content/Diffraction
!sudo apt-get update
!sudo apt-get install -y build-essential ninja-build

/content/Diffraction
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 1,581 B in 1s (1,275 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not

In [55]:
# Проверим среду выполнения и наличие GPU
import torch
print('GPU доступен:', torch.cuda.is_available())
!nvidia-smi || echo "GPU не обнаружен! Проверьте настройки среды выполнения."


GPU доступен: True
Tue Nov 18 12:08:38 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+----------------------------

## Настройка параметров CPU/CUDA решателей

Ниже можно выбрать значения `a`, `b`, `λ`, угла `θ`, числа коэффициентов `N` и глубины скин-слоя `δ`, которые будут прошиты в исходники `cpp_version/src/main.cpp` и `cuda_version/src/main.cu` перед сборкой. После изменения не забудьте повторно пересобрать нужную часть проекта, чтобы новые параметры попали в бинарники.


In [ ]:
# Виджет для выбора коэффициентов
import json
import math
import re
import subprocess
import sys
from pathlib import Path
from IPython.display import Markdown, display

try:
    import ipywidgets as widgets
except ModuleNotFoundError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "ipywidgets"])
    import ipywidgets as widgets


def _resolve_repo_root() -> Path:
    cwd = Path.cwd()
    if (cwd / "cpp_version").exists():
        return cwd
    alt_root = Path("/content/Diffraction")
    if alt_root.exists():
        return alt_root.resolve()
    return cwd


def _config_path(repo_root: Path) -> Path:
    content_root = Path("/content")
    if content_root.exists():
        return (content_root / "diffraction_params.json")
    return repo_root / "notebook_params.json"


def _ensure_angles(params: dict) -> dict:
    params = params.copy()
    if "theta_deg" not in params and "theta_rad" in params:
        params["theta_deg"] = math.degrees(params["theta_rad"])
    if "theta_rad" not in params:
        params["theta_rad"] = math.radians(params.get("theta_deg", 45.0))
    return params


def _fmt(value: float) -> str:
    return f"{value:.12g}"


def _update_source(path: Path, params: dict):
    pattern_map = [
        (r"const double a = [^;]+;", lambda p: f"const double a = {_fmt(p['a'])};"),
        (r"const double b = [^;]+;", lambda p: f"const double b = {_fmt(p['b'])};"),
        (r"const double lambda = [^;]+;", lambda p: f"const double lambda = {_fmt(p['lambda'])};"),
        (r"const double theta = [^;]+;", lambda p: f"const double theta = {_fmt(p['theta_rad'])};"),
        (r"const int N_default = [^;]+;", lambda p: f"const int N_default = {int(p['N'])};"),
        (r"const double skinDepth = [^;]+;", lambda p: f"const double skinDepth = {_fmt(p['delta'])};"),
    ]
    text = path.read_text()
    for pattern, repl_factory in pattern_map:
        text, count = re.subn(pattern, repl_factory(params), text, count=1)
        if count == 0:
            raise RuntimeError(f"Не удалось обновить {pattern} в {path}")
    path.write_text(text)


REPO_ROOT = _resolve_repo_root()
CONFIG_PATH = _config_path(REPO_ROOT)
DEFAULT_PARAMS = {
    "a": -1.0,
    "b": 1.0,
    "lambda": 1.0,
    "theta_deg": 45.0,
    "theta_rad": math.radians(45.0),
    "N": 10,
    "delta": 0.1,
}
params_state = _ensure_angles(DEFAULT_PARAMS)

if CONFIG_PATH.exists():
    try:
        loaded = json.loads(CONFIG_PATH.read_text())
        params_state.update(_ensure_angles(loaded))
    except Exception as exc:
        print("⚠️ Не удалось загрузить сохранённые параметры:", exc)

float_opts = dict(layout=widgets.Layout(width="220px"))
a_widget = widgets.FloatText(value=params_state["a"], description="a", **float_opts)
b_widget = widgets.FloatText(value=params_state["b"], description="b", **float_opts)
lambda_widget = widgets.FloatText(value=params_state["lambda"], description="λ", **float_opts)
theta_widget = widgets.FloatSlider(value=params_state["theta_deg"], min=0.0, max=85.0, step=0.5, description="θ (°)", readout_format=".1f")
N_widget = widgets.IntSlider(value=params_state["N"], min=4, max=200, step=2, description="N")
delta_widget = widgets.FloatText(value=params_state["delta"], description="δ", **float_opts)
apply_button = widgets.Button(description="Применить параметры", button_style="success")
status = widgets.Output()


def apply_params(_=None):
    params_state.update(
        _ensure_angles(
            {
                "a": float(a_widget.value),
                "b": float(b_widget.value),
                "lambda": float(lambda_widget.value),
                "theta_deg": float(theta_widget.value),
                "N": int(N_widget.value),
                "delta": float(delta_widget.value),
            }
        )
    )

    for rel in ("cpp_version/src/main.cpp", "cuda_version/src/main.cu"):
        _update_source(REPO_ROOT / rel, params_state)

    CONFIG_PATH.write_text(json.dumps(params_state, indent=2, ensure_ascii=False))

    with status:
        status.clear_output()
        display(Markdown(
            """**Параметры обновлены.**\\
a = {a:.3f}, b = {b:.3f}, λ = {lambda:.3f}, θ = {theta_deg:.1f}°, N = {N}, δ = {delta:.3f}""".format(**params_state)
        ))


def display_widget():
    info_label = widgets.HTML(value=f"<code>{CONFIG_PATH}</code>")
    return widgets.VBox([
        widgets.HBox([a_widget, b_widget, lambda_widget]),
        widgets.HBox([theta_widget, N_widget, delta_widget]),
        apply_button,
        status,
        widgets.HBox([widgets.Label("Config path:"), info_label]),
    ])

apply_button.on_click(apply_params)
display_widget()


In [ ]:
# Вставка ключевых участков исходников
from pathlib import Path
from IPython.display import Code, Markdown, display

REPO_ROOT = Path.cwd()
if not (REPO_ROOT / "cpp_version").exists():
    alt_root = Path("/content/Diffraction")
    if alt_root.exists():
        REPO_ROOT = alt_root.resolve()

sections = [
    {
        "title": "CPU: DifrOnLenta::SolveDifr()",
        "path": REPO_ROOT / "cpp_version/src/DifrOnLenta.cpp",
        "anchors": ["int DifrOnLenta::SolveDifr", "bool DifrOnLenta::SolveDifr"],
        "length": 160,
        "lang": "cpp",
    },
    {
        "title": "CPU: Гауссово решение матрицы",
        "path": REPO_ROOT / "cpp_version/src/Gauss.cpp",
        "anchors": ["int Gauss(", "bool CGauss::Solve"],
        "length": 140,
        "lang": "cpp",
    },
    {
        "title": "CUDA: DifrOnLentaCuda::SolveDifr()",
        "path": REPO_ROOT / "cuda_version/src/DifrOnLentaCuda.cu",
        "anchors": ["int DifrOnLentaCuda::SolveDifr", "bool DifrOnLentaCuda::SolveDifr"],
        "length": 160,
        "lang": "cuda",
    },
]


def _extract_snippet(path: Path, anchors, length: int) -> str:
    lines = path.read_text().splitlines()
    for anchor in anchors:
        for idx, line in enumerate(lines):
            if anchor in line:
                start = idx
                end = min(len(lines), idx + length)
                return "\n".join(lines[start:end])
    raise ValueError(f"Не найден ни один из якорей {anchors} в {path}")

for section in sections:
    try:
        snippet = _extract_snippet(section["path"], section["anchors"], section["length"])
    except ValueError as exc:
        display(Markdown(f"⚠️ {exc}"))
        continue
    display(Markdown(f"### {section['title']} ({section['path'].relative_to(REPO_ROOT)})"))
    display(Code(snippet, language=section["lang"]))


In [56]:
# Собираем и запускаем только CUDA-версию (выводит CSV)
!bash cuda_version/colab/build_cuda.sh




-- Configuring done (0.0s)
-- Generating done (0.0s)
-- Build files have been written to: /content/Diffraction/cuda_version/build
ninja: no work to do.

Running CUDA diffraction solver...

CUDA решатель дифракции на ленте

Решение с параметрами по умолчанию

Параметры:
a = -1, b = 1
λ = 1, θ = 45°
N = 10, δ = 0.1

Импедансный коэффициент χ: 0.628319 + 0.628319i

Коэффициенты Чебышева (10 шт):
#     Значение              
1     2.0673 + 0.6065i
2     4.1167 + -12.1461i
3     10.4708 + -7.6303i
4     -2.9418 + 21.6486i
5     -10.6767 + 5.1161i
6     -0.1652 + -9.5499i
7     4.7436 + -2.4744i
8     -1.6795 + 4.7446i
9     -1.7603 + -0.3503i
10    1.0604 + -0.3505i

Время выполнения:
Построение матрицы (GPU): 33.935 мс
Решение СЛАУ (CPU): 0.013 мс
Общее: 222.226 мс


Тестирование зависимости от N

N       GPU матрица (мс)CPU СЛАУ (мс)      Общее (мс) 
10      33.453                   0.011                    33.798            
20      51.882                   0.067                    52.29

In [ ]:
%cd /content
!if [ -d Diffraction ]; then rm -rf Diffraction; fi
!git clone https://github.com/dejar212/Diffraction.git
%cd Diffraction
!python cuda_version/colab/run_benchmark.py

/content
Cloning into 'Diffraction'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 95 (delta 2), reused 7 (delta 2), pack-reused 79 (from 1)
Receiving objects: 100% (95/95), 511.01 KiB | 17.62 MiB/s, done.
Resolving deltas: 100% (22/22), done.
/content/Diffraction
Проверка наличия CMake/compilers...

[cmd] cmake --version
cmake version 3.31.6

CMake suite maintained and supported by Kitware (kitware.com/cmake).

Установка python-библиотек для графиков...

[cmd] /usr/bin/python3 -m pip install --quiet matplotlib pandas


[cmd] nvidia-smi
Tue Nov 18 12:08:42 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | B

In [ ]:
# Синхронизация исходников с сохранёнными параметрами (запускайте после повторного клонирования)
import json
import math
import re
from pathlib import Path


def _resolve_repo_root() -> Path:
    cwd = Path.cwd()
    if (cwd / "cpp_version").exists():
        return cwd
    alt_root = Path("/content/Diffraction")
    if alt_root.exists():
        return alt_root.resolve()
    return cwd


def _config_path(repo_root: Path) -> Path:
    content_root = Path("/content")
    if content_root.exists():
        return content_root / "diffraction_params.json"
    return repo_root / "notebook_params.json"


def _ensure_angles(params: dict) -> dict:
    params = params.copy()
    if "theta_deg" not in params and "theta_rad" in params:
        params["theta_deg"] = math.degrees(params["theta_rad"])
    if "theta_rad" not in params:
        params["theta_rad"] = math.radians(params.get("theta_deg", 45.0))
    return params


def _fmt(value: float) -> str:
    return f"{value:.12g}"


def _update_source(path: Path, params: dict):
    pattern_map = [
        (r"const double a = [^;]+;", lambda p: f"const double a = {_fmt(p['a'])};"),
        (r"const double b = [^;]+;", lambda p: f"const double b = {_fmt(p['b'])};"),
        (r"const double lambda = [^;]+;", lambda p: f"const double lambda = {_fmt(p['lambda'])};"),
        (r"const double theta = [^;]+;", lambda p: f"const double theta = {_fmt(p['theta_rad'])};"),
        (r"const int N_default = [^;]+;", lambda p: f"const int N_default = {int(p['N'])};"),
        (r"const double skinDepth = [^;]+;", lambda p: f"const double skinDepth = {_fmt(p['delta'])};"),
    ]
    text = path.read_text()
    for pattern, repl_factory in pattern_map:
        text, count = re.subn(pattern, repl_factory(params), text, count=1)
        if count == 0:
            raise RuntimeError(f"Не удалось обновить {pattern} в {path}")
    path.write_text(text)


REPO_ROOT = _resolve_repo_root()
CONFIG_PATH = _config_path(REPO_ROOT)
if not CONFIG_PATH.exists():
    print("Сохранённые параметры не найдены — используются значения по умолчанию.")
else:
    params = _ensure_angles(json.loads(CONFIG_PATH.read_text()))
    for rel in ("cpp_version/src/main.cpp", "cuda_version/src/main.cu"):
        _update_source(REPO_ROOT / rel, params)
    print(f"Параметры из {CONFIG_PATH} применены к исходникам {REPO_ROOT}")


In [ ]:
%cd /content/Diffraction
!cmake -S cpp_version -B cpp_version/build_colab -G Ninja -DCMAKE_BUILD_TYPE=Release
!cmake --build cpp_version/build_colab --config Release
!cpp_version/build_colab/diffraction_solver > cpu_output.txt
!cat cpu_output.txt

In [ ]:
!cmake -S cuda_version -B cuda_version/build -G Ninja -DCMAKE_BUILD_TYPE=Release -DCMAKE_CUDA_ARCHITECTURES=75
!cmake --build cuda_version/build --config Release
!cuda_version/build/CudaDiffr > cuda_output.txt
!cat cuda_output.txt
!python cuda_version/colab/run_benchmark.py

In [ ]:
# Показывает пути к результатам и графикам
import os
print("CSV/JSON результаты:", os.listdir('cuda_version/plots'))
from IPython.display import Image
Image('cuda_version/plots/matrix_compare.png')


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('cuda_version/plots/combined_results.csv')
plt.figure(figsize=(10, 6))
plt.plot(df['N'], df['SolveTime_ms_CPU'], 'o-', label='СЛАУ CPU')
plt.plot(df['N'], df['SolveTime_ms_CUDA'], 's-', label='СЛАУ CUDA (cuSOLVER)')
plt.xlabel("N")
plt.ylabel("Время решения СЛАУ (мс)")
plt.title("Сравнение СЛАУ (CPU Gauss vs CUDA cuSOLVER)")
plt.legend()
plt.grid(alpha=0.3)
plt.show()


In [ ]:
# Графики ускорения (Speedup) для всех этапов
plt.figure(figsize=(12, 8))

speedup_matrix = df['MatrixTime_ms_CPU'] / df['MatrixGPU_ms_CUDA']
speedup_solve = df['SolveTime_ms_CPU'] / df['SolveTime_ms_CUDA']
speedup_total = df['TotalTime_ms_CPU'] / df['Total_ms_CUDA']

plt.plot(df['N'], speedup_solve, 'g^-', linewidth=2, label='Ускорение СЛАУ (CPU/GPU)')
plt.plot(df['N'], speedup_matrix, 'bs-', linewidth=2, label='Ускорение Матрицы (CPU/GPU)')
plt.plot(df['N'], speedup_total, 'ro-', linewidth=2, label='Ускорение Всего (CPU/GPU)')

plt.xlabel("N (число базисных функций)")
plt.ylabel("Ускорение (раз)")
plt.title("Эффективность GPU ускорения")
plt.legend()
plt.grid(True, alpha=0.3)
plt.yscale('log')  # Логарифмическая шкала, так как ускорение может быть огромным
plt.show()